In [1]:
import random

import torch

In [2]:
device = torch.device("mps")

In [4]:
device

device(type='mps')

In [4]:
def inv_sphere_proj(raw_point, n_dim, n_train, radius):
    """inverse stereographic projection

    n_dim:
    the dimension of the original data

    n_train:
    the number of points to be projected

    radius:
    the radius of the sphere
    """

    res = []
    for i in range(n_train):
        tmp = []
        normal = torch.sum(torch.square(raw_point[i]))
        for j in range(n_dim):
            tmp.append(torch.sum((2 * raw_point[i][j] / (normal + 1) * radius)))
        tmp.append(torch.sum(((normal - 1) / (normal + 1) * radius)))
        res.append(torch.stack(tmp))
    res = torch.stack(res)
    return res

In [8]:
inv_sphere_proj(torch.Tensor([[1,1]]), 2, 1, 1)

tensor([[-0.6667,  0.6667,  0.3333]])

In [9]:
import math

In [17]:
def get_g_loss(fake):
    # return tf.reduce_mean(1. - fake)
    return -torch.mean(torch.log(fake + 1e-10))

In [25]:
get_g_loss(torch.Tensor([[2,1],[1,2]]))

tensor(-0.3466)

In [22]:
torch.log(torch.Tensor([1,2]))

tensor([0.0000, 0.6931])

In [27]:
import torch.nn as nn

In [34]:
class Discriminator(nn.Module):
    def __init__(self, stride=1):
        super(Discriminator, self).__init__()
        self.dense1 = nn.Linear(in_features=3, out_features=5)

    def forward(self, x):
        out = self.dense1(x)
        return out

In [35]:
net = Discriminator()

In [40]:
a = torch.Tensor([[-3.9648,  1.4630,  1.3172],
        [-0.9370,  2.6130,  1.3896],
        [-1.1344,  0.9756,  0.6380]])

In [41]:
a

tensor([[-3.9648,  1.4630,  1.3172],
        [-0.9370,  2.6130,  1.3896],
        [-1.1344,  0.9756,  0.6380]])

In [42]:
net(a)

tensor([[ 1.7146, -1.3754, -1.0005, -2.1532, -1.7996],
        [-0.1205, -0.0937, -1.0022, -1.9742,  0.1841],
        [ 0.3528, -0.2894, -0.4152, -1.2566, -0.4335]],
       grad_fn=<AddmmBackward0>)

TypeError: new() received an invalid combination of arguments - got (list, dtype=torch.dtype), but expected one of:
 * (*, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [45]:
torch.has_mps

True

In [46]:
net.to("mps")

Discriminator(
  (dense1): Linear(in_features=3, out_features=5, bias=True)
)

In [ ]:
Tensor = torch.cuda.FloatTensor

In [27]:
from scipy.integrate import dblquad
import numpy as np
import math
import torch

In [22]:
u_G = np.random.random(5)
assert (u_G == 0).sum() != 10
radius = math.sqrt((u_G * u_G).sum())
tmp = np.random.random((5, 5))
tmp = np.dot(tmp, tmp.transpose())
sig_G = tmp + tmp.T

In [24]:
sig_G

array([[4.82893791, 3.99940947, 4.6485405 , 2.79303387, 4.29967256],
       [3.99940947, 5.74339077, 6.32683091, 3.83662554, 4.27836264],
       [4.6485405 , 6.32683091, 7.29596813, 4.13152304, 4.68629953],
       [2.79303387, 3.83662554, 4.13152304, 2.63390093, 2.9847854 ],
       [4.29967256, 4.27836264, 4.68629953, 2.9847854 , 4.21438484]])

In [16]:
def gauss(x, sigma):
    return np.exp(-np.dot(np.dot(x, np.linalg.inv(sigma)), x) / 2) / (2 * np.pi) / np.sqrt(np.linalg.det(sigma))

In [23]:
def cal_numerical_TVD(sigma, i, j, c):
    sig = np.array([[sigma[i][i], sigma[i][j]], [sigma[j][i], sigma[j][j]]])
    print(sig)
    val, err = dblquad(lambda y, x: np.abs(gauss(np.array([x, y]), sig) - gauss(np.array([x, y]), c * sig)) / 2,
                       -np.inf, np.inf, -np.inf, np.inf)
    return val

In [25]:
cal_numerical_TVD(sig_G, 1, 4, 2)

[[5.74339077 4.27836264]
 [4.27836264 4.21438484]]


0.24999995810045472

In [30]:
from torch.autograd import Variable

In [4]:
Tensor = torch.FloatTensor

In [43]:
test_ori_data = Variable(Tensor(np.random.multivariate_normal(np.zeros(5), (1 + 1) * sig_G, 3)))
test_ori = torch.cat((test_ori_data[:, 1:2], test_ori_data[:, 2:3]), dim=1)

In [44]:
test_ori_data

tensor([[-4.8529,  0.6363, -0.1105, -0.0201, -2.3349],
        [ 1.2353,  1.9969,  3.0469,  0.9457,  1.0823],
        [-0.8795,  0.3101, -1.2183,  0.6472,  0.8665]])

In [45]:
test_ori

tensor([[ 0.6363, -0.1105],
        [ 1.9969,  3.0469],
        [ 0.3101, -1.2183]])

In [39]:
test_ori_data[:, 1:2]

tensor([[-3.0260],
        [ 3.1249],
        [-0.7521]])

In [58]:
import random
import torch.nn as nn

In [59]:
random.randint(0, 1)

1

In [60]:
[ 0.84623921 0.15091511 0.12822951 0.71534105 0.93871608 0.17218971 0.55273356 0.88380656 0.35630968 0.9762188 ]

[ 0.8607, -0.1706, -0.6923,  0.7893,  0.9136, -0.5567,  0.2506,  0.6171, -0.1277,  0.9316]

[ 0.9276, -0.0623, -0.5001,  0.9249,  0.9106, -0.1990,  0.4993,  0.5846, 0.2723,  0.9168]

SyntaxError: invalid syntax (1067259103.py, line 2)

In [61]:
test_sample = 100000
c = 1

In [62]:
point = Variable(Tensor([ 0.8582,  0.0032, -0.2556,  0.8472,  0.8838, -0.1246,  0.4288,  0.6623, 0.2847,  1.1510]))
u_G = [0.84623921, 0.15091511, 0.12822951, 0.71534105, 0.93871608, 0.17218971, 0.55273356, 0.88380656, 0.35630968, 0.9762188]


In [63]:
class test_Discriminator(nn.Module):
    def __init__(self):
        super(test_Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(2, 64),
            nn.LeakyReLU(0.8, inplace=True),
            nn.Linear(64, 256),
            nn.LeakyReLU(0.8, inplace=True),
            nn.Linear(256, 256),
            nn.LeakyReLU(0.8, inplace=True),
            nn.Linear(256, 64),
            nn.LeakyReLU(0.8, inplace=True),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        validity = self.model(x)
        return validity

In [64]:
test_dis = test_Discriminator().to(device)
optimizer_testD = torch.optim.Adam(test_dis.parameters(), lr=0.01)

NameError: name 'device' is not defined

In [65]:
def gen_g_data(exp, cov, n_sample=100000):
    ori = np.random.multivariate_normal(exp, cov, n_sample)
    return ori

In [66]:
def get_d_loss(net, real, fake):
    # return -tf.reduce_mean(real) - tf.reduce_mean(1. - fake)
    return -torch.mean(torch.log(net(real) + 1e-10) + torch.log(1. - net(fake) + 1e-10))

def get_g_loss(fake):
    # return tf.reduce_mean(1. - fake)
    return -torch.mean(torch.log(fake + 1e-10))

def d_real_loss(real, fake):
    """
    calculate the expectation of
        real + 1 - fake

    :param real:
    :param fake:
    :return:
    """
    return torch.mean(real) + torch.mean(1. - fake)

def d_metric(net, ori, contrast):
    # get the probabilities
    ori = net(ori)
    contrast = net(contrast)

    # get D^
    ori = torch.where(ori < 0.5, 0., 1.)
    contrast = torch.where(contrast < 0.5, 0., 1.)

    # print calculated TVD
    metric = d_real_loss(ori, contrast) - 1
    # tf.print("TVD-ori", metric, "TVD-cal", a, "delta", tf.abs(metric - a))
    return metric

def g_metric_base(c, n_dim):
    """
    TV distance 解析解

    c:

    n_dim:
        dimension of the data
    """
    r_sq = n_dim * (1 + c) * np.log(1 + c) / c
    return torch.igamma(torch.tensor(n_dim) / 2, torch.tensor(r_sq) / 2) - torch.igamma(torch.tensor(n_dim) / 2, torch.tensor(r_sq) / ((1 + c) * 2))

In [67]:
point = torch.tile(point, (test_sample, 1))
print("ug", u_G, "point", point[0])
# --------------------------
# Test
# --------------------------
print("------------------------------Test-result-----------------------------------------")
for test in range(6):
    dim_1 = random.randint(0, 9)
    dim_2 = random.randint(0, 9)
    while dim_1 == dim_2:
        dim_2 = random.randint(0, 9)
    TVD_num = cal_numerical_TVD(sig_G, dim_1, dim_2, c + 1)
    print("trial", test, "dim", dim_1, dim_2, "TVD", TVD_num)
    for i in range(50):
        test_ori_data = Variable(Tensor(gen_g_data(u_G, sig_G, test_sample))).to(device)
        test_fake_data = point + Variable(Tensor(np.random.multivariate_normal(np.zeros(10), (c + 1) * sig_G, test_sample))).to(device)

        test_ori = torch.cat((test_ori_data[:, dim_1:dim_1 + 1], test_ori_data[:, dim_2:dim_2 + 1]), dim=1)
        test_fake = torch.cat((test_fake_data[:, dim_1:dim_1 + 1], test_fake_data[:, dim_2:dim_2 + 1]), dim=1)
        # print(test_ori, test_fake)

        prev_metric = 1000.
        for d_epoch in range(5000):
            optimizer_testD.zero_grad()

            d_loss = get_d_loss(test_dis, test_ori, test_fake.detach())
            d_loss.backward()
            optimizer_testD.step()
            train_metric = d_metric(test_dis, test_ori, test_fake)
            if (torch.abs_((train_metric - prev_metric)) < 1e-4):
                print(train_metric.item())
                break
            prev_metric = train_metric

ug [0.84623921, 0.15091511, 0.12822951, 0.71534105, 0.93871608, 0.17218971, 0.55273356, 0.88380656, 0.35630968, 0.9762188] point tensor([ 0.8582,  0.0032, -0.2556,  0.8472,  0.8838, -0.1246,  0.4288,  0.6623,
         0.2847,  1.1510])
------------------------------Test-result-----------------------------------------


IndexError: index 7 is out of bounds for axis 0 with size 5

In [79]:
A = [[7.6224201,  5.13687972, 6.20101875, 5.23219945, 5.50632698, 7.71639942, 4.37488368, 6.59550037, 5.85394109, 4.5853117 ],
 [5.13687972, 6.55478527, 3.85091955, 3.96130058, 5.74537182, 6.6014608, 3.15404838, 4.50551946, 6.40139768, 3.86376401],
 [6.20101875, 3.85091955, 6.58735655, 5.10539876, 4.60607361, 6.61517404, 4.07005041, 6.07003765, 5.54842133, 2.89232711],
 [5.23219945, 3.96130058, 5.10539876, 5.19042475, 4.93365137, 6.30952044, 3.88081521, 4.38545165, 5.14069357, 3.19297651],
 [5.50632698, 5.74537182, 4.60607361, 4.93365137, 6.23173784, 7.0182592, 4.08759713, 4.08465763, 5.95464345, 3.93888992],
 [7.71639942, 6.6014608,  6.61517404, 6.30952044, 7.0182592,  9.24235444, 4.85003648, 6.08668315, 7.43283991, 4.99179672],
 [4.37488368, 3.15404838, 4.07005041, 3.88081521, 4.08759713, 4.85003648, 3.93210023, 3.60741328, 3.90110952, 2.88819139],
 [6.59550037, 4.50551946, 6.07003765, 4.38545165, 4.08465763, 6.08668315, 3.60741328, 7.7184,     5.9526708,  3.42800809],
 [5.85394109, 6.40139768, 5.54842133, 5.14069357, 5.95464345, 7.43283991, 3.90110952, 5.9526708,  8.01019363, 3.96765813],
 [4.5853117,  3.86376401, 2.89232711, 3.19297651, 3.93888992, 4.99179672, 2.88819139, 3.42800809, 3.96765813, 3.89080807]]

In [ ]:
[0.68642877 0.90400085 0.89764726 0.70365138 0.63324737 0.36242745 0.3663052  0.85856962 0.34622337 0.66512134]

In [72]:
u_G = np.random.random(10)
assert (u_G == 0).sum() != 10
radius = math.sqrt((u_G * u_G).sum())
tmp = np.random.random((10, 10))
tmp = np.dot(tmp, tmp.transpose())
sig_G = tmp + tmp.T
print(u_G, sig_G)

[0.57398805 0.34198136 0.64399551 0.38267723 0.34367104 0.14869019
 0.79870514 0.67632847 0.1318016  0.59037254] [[3.30449    1.69253706 3.05125322 3.60404035 3.0630276  4.09816304
  1.77090245 2.76432    2.44814641 3.03077962]
 [1.69253706 3.11966672 3.96368838 4.12134094 4.26233891 3.52004754
  2.995824   2.61146224 3.0736474  3.54939428]
 [3.05125322 3.96368838 8.80338246 6.36310199 6.07468609 5.75921539
  6.35196818 3.94752103 4.38798945 4.90481352]
 [3.60404035 4.12134094 6.36310199 7.6685419  6.26540191 5.6088127
  4.94724775 5.33062349 4.54211857 5.58198135]
 [3.0630276  4.26233891 6.07468609 6.26540191 7.87390595 6.33380078
  5.03710417 4.68790592 5.38756504 6.11887307]
 [4.09816304 3.52004754 5.75921539 5.6088127  6.33380078 7.62930541
  4.56801057 4.18924386 4.9372153  4.99196554]
 [1.77090245 2.995824   6.35196818 4.94724775 5.03710417 4.56801057
  5.81987104 3.41310165 3.28185439 3.983062  ]
 [2.76432    2.61146224 3.94752103 5.33062349 4.68790592 4.18924386
  3.41310165 4.

In [82]:
A = [[6.70344745, 3.72302146, 5.12721976, 3.45682491, 2.50684365, 4.50200168,5.35977539, 3.91405633, 3.12108987, 3.66437961],
 [3.72302146, 4.71241606, 3.84008895, 2.41718105, 2.8675421,  4.31742486,3.114784, 3.88741647, 1.93085409, 3.62538204],
 [5.12721976, 3.84008895, 6.59543031, 4.51623981, 3.35127452, 5.65210835,4.36272335, 5.03369366, 3.04455897, 3.72150854],
 [3.45682491, 2.41718105, 4.51623981, 3.59241169, 2.15913668, 4.07915255,2.90886253, 3.05325609, 2.18920194, 2.36495133],
 [2.50684365, 2.8675421  ,3.35127452, 2.15913668, 4.09798389, 4.52556228,2.35404212, 3.64482338 ,1.49189422 ,3.50601166],
 [4.50200168, 4.31742486, 5.65210835, 4.07915255, 4.52556228, 7.52802586,4.20885642, 6.0450614,  2.757288  , 4.17512582],
 [5.35977539, 3.114784,   4.36272335, 2.90886253, 2.35404212, 4.20885642,4.91019135, 4.05200586, 2.26395103, 3.24743557],
 [3.91405633, 3.88741647, 5.03369366, 3.05325609, 3.64482338, 6.0450614,4.05200586, 6.60162526, 2.05166513, 3.34668222],
 [3.12108987, 1.93085409, 3.04455897, 2.18920194, 1.49189422, 2.757288,2.26395103, 2.05166513, 2.16708823, 1.94724058],
 [3.66437961, 3.62538204, 3.72150854, 2.36495133, 3.50601166, 4.17512582,3.24743557, 3.34668222, 1.94724058, 4.15420753]]

In [43]:
import numpy as np

In [44]:
B = np.linalg.eigvals(A)

In [45]:
B

array([59.12036639,  7.52811779,  6.84795035,  7.30276283,  5.8388788 ,
        4.76385161,  4.00475238,  4.04759134,  4.22050916,  4.53072454])

In [40]:
A = [[10.76436003,  5.47605482 , 4.52605946 , 4.87705617 , 5.25142921 , 5.56086034,
   3.9250431,   4.60694116 , 4.80677584 , 6.30234086],
 [ 5.47605482, 10.17185045 , 5.18466563 , 4.96431168 , 5.21379797 , 5.36460511,
   5.42815691,  4.84284419,  5.25406816 , 6.20031935],
 [ 4.52605946,  5.18466563 ,10.60027366 , 3.92397824 , 4.502657,    5.0143747,
   4.7175623,   5.11881256 , 5.0154492 ,  6.2571088 ],
 [ 4.87705617,  4.96431168 , 3.92397824 , 9.69087419 , 5.92660257,  5.43168463,
   4.66315653,  4.90088661 , 4.87544232 , 5.39532108],
 [ 5.25142921,  5.21379797 , 4.502657  ,  5.92660257 ,11.3983297,   6.00013756,
   4.73375662,  6.37893814 , 5.86581683 , 6.7452099 ],
 [ 5.56086034,  5.36460511 , 5.0143747  , 5.43168463 , 6.00013756, 10.19875276,
   5.19244488,  5.8629758  , 5.38952709,  6.21553577],
 [ 3.9250431 ,  5.42815691,  4.7175623 ,  4.66315653 , 4.73375662,  5.19244488,
  10.79174575,  4.8435046 ,  4.83713841,  4.87742913],
 [ 4.60694116,  4.84284419 , 5.11881256 , 4.90088661 , 6.37893814 , 5.8629758,
   4.8435046,  10.74937463  ,5.54911376,  6.38249321],
 [ 4.80677584 , 5.25406816 , 5.0154492,   4.87544232 , 5.86581683 , 5.38952709,
   4.83713841 , 5.54911376 ,11.17661831 , 7.46287822],
 [ 6.30234086 , 6.20031935 , 6.2571088  , 5.39532108 , 6.7452099 ,  6.21553577,
   4.87742913 , 6.38249321 , 7.46287822, 12.6633257 ]]

In [ ]:
[0.09794686 0.25410163 0.38754689 0.47619852 0.27091234 0.29948665 0.97327622 0.00507328 0.93313256 0.27981733]

In [49]:
inputs = Tensor([ 6.30234086 , 6.20031935 , 6.2571088  , 5.39532108 , 6.7452099 ,  6.21553577,
   4.87742913 , 6.38249321 , 7.46287822, 12.6633257 ])

In [52]:
inputs

tensor([ 6.3023,  6.2003,  6.2571,  5.3953,  6.7452,  6.2155,  4.8774,  6.3825,
         7.4629, 12.6633])

In [51]:
inputs.numpy()

array([ 6.302341 ,  6.2003193,  6.2571087,  5.395321 ,  6.7452097,
        6.2155356,  4.877429 ,  6.382493 ,  7.462878 , 12.663325 ],
      dtype=float32)

In [7]:
torch.zeros(1, 5, dtype=torch.float32)

tensor([[0., 0., 0., 0., 0.]])

In [10]:
import torch.nn as nn

In [11]:
nn.Parameter(torch.zeros(1, 5, dtype=torch.float32))

Parameter containing:
tensor([[0., 0., 0., 0., 0.]], requires_grad=True)

In [14]:
class NN2(nn.Module):
  def __init__(self, input_shape, n_width=32, n_out=None, **kwargs):
    super(NN2, self).__init__(**kwargs)
    self.n_width = n_width
    self.n_out = n_out

    self.l_1 = nn.Linear(input_shape, self.n_width)
    self.l_2 = nn.Linear(self.n_width, self.n_width)

    n_out = self.n_out or int(input_shape[-1])
    self.l_f = nn.Linear(self.n_width, n_out)

  def call(self, inputs):
    # relu with low regularity
    x = nn.ReLU(self.l_1(inputs))
    x = nn.ReLU(self.l_2(x))

    # tanh with high regularity
    #x = tf.nn.tanh(self.l_1(inputs))
    #x = tf.nn.tanh(self.l_2(x))

    x = self.l_f(x)

    return x

In [17]:
NN2([3,5],5, (3))

TypeError: empty(): argument 'size' must be tuple of SymInts, but found element of type list at pos 2

In [20]:
r=1.2
bound = 30
n_bins = 4

In [29]:
m = n_bins/2
x1L = bound*(r-1.0)/(torch.pow(r, Tensor([m]))-1.0)

index = torch.reshape(torch.range(0, n_bins+1, dtype=torch.float32),(-1,1))
index -= m
xr = torch.where(index>=0, (1.-torch.pow(r, Tensor(index)))/(1.-r),
              (1.-torch.pow(r,torch.abs(index)))/(1.-r))
xr = torch.where(index>=0, x1L*xr, -x1L*xr)
xr = torch.reshape(xr,(-1,1))
xr = (xr + bound)/2.0/bound

/var/folders/vm/7scsvf852h1csts2mztj6hnr0000gn/T/ipykernel_9123/2994907521.py:4: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  index = torch.reshape(torch.range(0, n_bins+1, dtype=torch.float32),(-1,1))


In [31]:
a = torch.concat([torch.reshape(Tensor([0.0]),(-1,1)), torch.reshape(xr[1:-1,0],(-1,1)), torch.reshape(Tensor([1.0]),(-1,1))],0)
a

tensor([[0.0000],
        [0.2727],
        [0.5000],
        [0.7273],
        [1.0000],
        [1.0000]])

In [32]:
torch.tile(a, [1, 5])

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2727, 0.2727, 0.2727, 0.2727, 0.2727],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.7273, 0.7273, 0.7273, 0.7273, 0.7273],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])

In [37]:
torch.empty((3, 4), dtype=torch.float32)

tensor([[0.0000e+00, 0.0000e+00, 1.4013e-45, 0.0000e+00],
        [1.4013e-45, 0.0000e+00, 1.4013e-45, 0.0000e+00],
        [1.4013e-45, 0.0000e+00, 1.4013e-45, 0.0000e+00]])

In [39]:
nn.Parameter(torch.zeros(1, 5, dtype=torch.float32))

Parameter containing:
tensor([[0., 0., 0., 0., 0.]], requires_grad=True)

In [68]:
a = [0.85169915, 0.2991933 , 0.87462798 ,0.36485074 ,0.34229901, 0.8980613 , 0.50002345, 0.00633536, 0.16558528 ,0.03962351]
a = torch.tensor(a)
1.2175043/torch.norm(a,dim=0,p=2).item()

0.7130116688930774

In [2]:
import torch

In [10]:
a = [1.0593, 0.8722, 0.9306, 1.0661, 0.8598, 0.9132, 1.1765, 0.9211, 1.0185,1.1277]
a = torch.tensor(a) - 1

In [12]:
torch.norm(a,dim=0,p=2).item()

0.332042396068573

In [128]:
a.type(torch.IntTensor)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)

In [102]:
a.size()[0]

10

In [2]:
import krmodel.BR_model as BR_model

In [4]:
net = BR_model.IM_rNVP_KR_CDF(2,32,8)

TypeError: reshape(): argument 'input' (position 1) must be Tensor, not list

In [ ]:
torch.BoolTensor

In [113]:
(~torch.BoolTensor([[True,False],[False,True]])).float()

tensor([[0., 1.],
        [1., 0.]])

In [2]:
import numpy as np

In [121]:
len(torch.zeros_like(torch.BoolTensor([[True,False],[False,True]])).shape)

2

In [132]:
x = np.random.random((200,2))

In [133]:
net(x)

TypeError: __init__() missing 2 required positional arguments: 'n_step' and 'n_depth'